# Performance of Models: FFT

![Old BOA](../plots/notebook_plots/model_goodness_of_fit/fft/old.png "Old BOA")
![New BOA](../plots/notebook_plots/model_goodness_of_fit/fft/new.png "New BOA")

# Performance of Models: Stencil3D

![Old BOA](../plots/notebook_plots/model_goodness_of_fit/stencil3d/old.png "Old BOA")
![New BOA](../plots/notebook_plots/model_goodness_of_fit/stencil3d/new.png "New BOA")

# Performance of Models: GEMM

![New BOA](../plots/notebook_plots/model_goodness_of_fit/gemm/new.png "New BOA")

# BayesOpt Performance: Stencil3D

![New BOA](../plots/notebook_plots/bayesopt_analysis/stencil3d/bayesopt.png "New BOA")
![New BOA](../plots/notebook_plots/bayesopt_analysis/stencil3d/bayesopt_comparisons.png "New BOA")

# BayesOpt Performance: GEMM

![New BOA](../plots/notebook_plots/bayesopt_analysis/gemm/bayesopt.png "New BOA")

# Learning the Ordering: Method

**Perspective:** for $N$ outputs, there are $N!$ GPAR models we could choose from: **model selection**!

Classic model selection framework: 
- Split data into **train**, **validation**, **test**.
- Select select model on **validation**
- Retrain on **train + validation**, report performance on **test** 

Our experimental setup:

1. Split data into **train+validation** and **test**.
2. Repeat the steps below $E$ times:
3. Split get a **train** and **validation** split.
4. Consider $P$ permutations.
5. Train $P$ models, and report their performances on **validation**
6. Pick best model, retrain on **train+validation**, report performance on **test**.

Both for random and greedy search, we ran $E = 25$ experiments.

- For **random** search, we sampled $P = 9$ permutations uniformly.
- For **greedy** search, $P = \frac{9(9 - 1)}{2} = 36$ permutations are considered (by following greedy selection).

# Learning the Ordering: Hierarchical BayesOpt

## Motivation

**Random search**: Uniformly sample $P$ permutations, train $P$ GPAR models, select best permutation based on validation set log-likelihood.

**Problem:** GPAR training expensive, sampling doesn't take into account previous observations.

**Solution:** Perform BO to select the permutation!

## Details

**To perform BO, we need a kernel over permutations.**

**Idea:** Modify the EQ kernel appropriately.

EQ kernel:

$$
    k(x, y) = \alpha^2 \exp\left\{\frac{||x - y||^2}{L}\right\}
$$

Assume $x, y \in \mathbb{R}^N$, and $x, y \sim \mathcal{N}(0, I)$. Then

$$
    \mathbb{E}||x - y||^2 = 2N (1 - \text{Corr}[x, y]),
$$
where $\text{Corr}[x, y]$ is the Pearson correlation.

Kendall $\tau$: Correlation coefficient between two permutations $\pi$ and $\rho$ over $N$ items:
$$
    \tau(\pi, \rho) = 1 - \frac{2d(\pi, \rho)}{N(N - 1)}
$$

Perform replacement:
$$
    k(x, y) = \alpha^2 \exp\left\{\frac{||x - y||^2}{L}\right\} \quad\to\quad
    k'(\pi, \rho) = \alpha^2 \exp\left\{\frac{2N (1 - \tau(\pi, \rho))}{L}\right\} = \alpha^2 \exp\left\{\frac{2d(\pi, \rho)}{L(N - 1)}\right\}
$$

**Note:** $k'(\pi, \rho)$ is clearly a valid kernel!

This is a single-output problem: Use **Expected Improvement** as our acquisition function.

# Learning the Ordering: Baselines on Stencil3D

![New BOA](../plots/notebook_plots/ordering/stencil3d/random_vs_greedy.png?3 "New BOA")